In [3]:
import os
import numpy as np
import cv2
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage
from PIL import Image

# Load augmented dataset
X_aug = np.load("X_fingerprint_aug.npy")  # Shape: (num_augmented_images, 128, 128, 1)
y_aug = np.load("y_fingerprint_aug.npy")  # Labels

# Path to save segmented images
segmented_dir = r"D:\segmented_data"
os.makedirs(segmented_dir, exist_ok=True)

segmented_X = []

print(f"Segmenting {len(X_aug)} images...")

for i, img in enumerate(X_aug):
    img = (img * 255).astype(np.uint8).squeeze()  # Convert to uint8 and remove channel dimension

    # Apply Otsu's threshold
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Compute the distance transform
    D = ndimage.distance_transform_edt(thresh)

    # Find local maxima (new version fix)
    coordinates = peak_local_max(D, min_distance=10, labels=thresh)
    local_max = np.zeros_like(D, dtype=bool)
    local_max[tuple(coordinates.T)] = True

    # Perform connected component analysis
    markers, _ = ndimage.label(local_max)

    # Apply Watershed
    labels = watershed(-D, markers, mask=thresh)

    # Convert to binary mask (Segmented image)
    segmented = np.where(labels > 0, 255, 0).astype(np.uint8)

    # Save segmented image
    segmented_X.append(segmented)
    save_path = os.path.join(segmented_dir, f"seg_{i}.png")
    Image.fromarray(segmented).save(save_path)

    if i % 500 == 0:
        print(f"Processed {i}/{len(X_aug)} images...")

# Convert to NumPy array and normalize
segmented_X = np.array(segmented_X) / 255.0  # Normalize to [0,1]

# Save segmented dataset
np.save("X_fingerprint_segmented.npy", segmented_X)
np.save("y_fingerprint_segmented.npy", y_aug)

print(f"✅ Segmentation completed! Total segmented images: {len(segmented_X)}")


Segmenting 24000 images...
Processed 0/24000 images...
Processed 500/24000 images...
Processed 1000/24000 images...
Processed 1500/24000 images...
Processed 2000/24000 images...
Processed 2500/24000 images...
Processed 3000/24000 images...
Processed 3500/24000 images...
Processed 4000/24000 images...
Processed 4500/24000 images...
Processed 5000/24000 images...
Processed 5500/24000 images...
Processed 6000/24000 images...
Processed 6500/24000 images...
Processed 7000/24000 images...
Processed 7500/24000 images...
Processed 8000/24000 images...
Processed 8500/24000 images...
Processed 9000/24000 images...
Processed 9500/24000 images...
Processed 10000/24000 images...
Processed 10500/24000 images...
Processed 11000/24000 images...
Processed 11500/24000 images...
Processed 12000/24000 images...
Processed 12500/24000 images...
Processed 13000/24000 images...
Processed 13500/24000 images...
Processed 14000/24000 images...
Processed 14500/24000 images...
Processed 15000/24000 images...
Proce